In [6]:
import os
import shapely
import pdal
import geopandas as gpd

In [7]:
gdf = gpd.read_file('boundaries/resources.geojson')

In [8]:
request = gpd.GeoSeries(shapely.geometry.Polygon.from_bounds(-111.8754868549115002,35.0912043389367412, -111.2376941285862557,35.6177805400308856),crs=4326).to_crs(gdf.crs)

In [9]:
request = gpd.read_file('E:/PipelineFire/tunnel_bounds.geojson')

In [10]:
match = gpd.clip(gdf,request)

In [11]:
urls = list(match['url'])

In [12]:
import pypipe
import json

In [13]:
urls = [urls[2]]

IndexError: list index out of range

In [14]:
bounds = pypipe.stringify_bounds(request.to_crs(3857).total_bounds)

if len(urls) == 0:
    raise RuntimeError('Could not find point cloud that overlaps request')

elif len(urls) == 1:
    url = urls[0]
    pipe = [
        pypipe.readers.ept(url,bounds=bounds),
        pypipe.writers.las('download.las')
    ]
elif len(urls) > 1:
    pipe = [pypipe.readers.ept(url,bounds=bounds) for url in urls]
    pipe.append(pypipe.filters.merge())
    pipe.append(pypipe.writers.las('download.las'))

#pipe = pdal.Pipeline(json.dumps(pipe))
#pipe.execute()

In [8]:
pipe.metadata

{'metadata': {'filters.merge': {},
  'readers.ept': [{'comp_spatialreference': 'PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Mercator_1SP"],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["X",EAST],AXIS["Y",NORTH],EXTENSION["PROJ4","+proj=merc +a=6378137 +b=6378137 +lat_ts=0.0 +lon_0=0.0 +x_0=0.0 +y_0=0 +k=1.0 +units=m +nadgrids=@null +wktext +no_defs"],AUTHORITY["EPSG","3857"]]',
    'spatialreference': 'PROJCS["WGS 84 / Pseudo-Mercator",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degr

In [11]:
urls

['https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_AZ_VerdeKaibab_B1_2018_LAS_2019/ept.json',
 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/USGS_LPC_AZ_VerdeKaibab_B2_2018_LAS_2019/ept.json',
 'https://s3-us-west-2.amazonaws.com/usgs-lidar-public/AZ_Coconino_B1_2019/ept.json']

In [21]:
class Pipeline(list):
    def __init__(self,list):
        super().__init__(list)

    def to_json_string(self):
        import json
        return json.dumps(self)

    def to_json_file(self,path):
        with open(path,'w') as f:
            f.write(json.dumps(self))

    def to_json_tempfile(self):
        import tempfile
        file = tempfile.NamedTemporaryFile().name
        self.to_json_file(file)
        return file

    def execute(self):
        import pdal
        pipe = pdal.Pipeline(self.to_json_string())
        pipe.execute()
        return pipe

    def execute_subprocess(self):
        import subprocess
        file = self.to_json_tempfile()
        subprocess.run('pdal','pipeline',file)

In [17]:
p = Pipeline(pipe)

In [19]:
p.to_json_string()

'[{"type": "readers.ept", "filename": "https://s3-us-west-2.amazonaws.com/usgs-lidar-public/AZ_Coconino_B1_2019/ept.json", "bounds": "([-12409659.723118749,-12401014.651463743],[4210651.769188692,4218212.139389493])"}, {"type": "writers.las", "filename": "download.las"}]'

In [20]:
p.to_json_file('test.json')